In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import config
from config import username, api_key
plotly.tools.set_credentials_file(username=username, api_key=api_key)
import plotly.plotly as py

data = pd.read_csv("graddownload2017f.csv", encoding='ISO-8859-1')
fips_finder = pd.read_csv("all-geocodes-v2016.csv", encoding='ISO-8859-1')

In [97]:
#Get only needed columns, add "county" to county name
less_data = data[["schoolname", "countyname", "currstatus"]].dropna(how='any')
less_data.loc[:,"countyname"] = less_data["countyname"].astype(str) + " County"

#Group by county, take the mean
grouped_data = less_data.groupby(["countyname"])
mean_grad = grouped_data.mean().reset_index()
mean_grad.head()

,countyname,currstatus
0,Alameda County,84.838517
1,Amador County,89.750000
2,Butte County,84.470930
3,Calaveras County,91.747619
4,Colusa County,89.169565


In [98]:
#Add leading zeroes to FIPS
fips_finder.loc[:,"State Code (FIPS)"] = fips_finder["State Code (FIPS)"].map("{:02d}".format)
fips_finder.loc[:,"County Code (FIPS)"] = fips_finder["County Code (FIPS)"].map("{:003d}".format)
fips_finder.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,00,000,0,0,0,United States
1,40,01,000,0,0,0,Alabama
2,50,01,001,0,0,0,Autauga County
3,50,01,003,0,0,0,Baldwin County
4,50,01,005,0,0,0,Barbour County


In [99]:
#Add full FIPS to new column
fips_renamed = fips_finder.rename(columns={"Area Name (including legal/statistical area description)":"countyname"})
fips_renamed["FIPS"] = fips_renamed["State Code (FIPS)"].map(str) + fips_renamed["County Code (FIPS)"].map(str)
fips_less = fips_renamed[["countyname", "FIPS"]]

#Keep only CA data
fips_ca = fips_less[fips_less["FIPS"].str[:2] == "06"]
fips_ca.head()

#Export to CSV for future use
fips_ca.to_csv("FIPS_CA.csv", index=False)

In [100]:
merge_table = pd.merge(mean_grad, fips_ca, on="countyname")

#Export to CSV for comparison
merge_table.to_csv("HS_Grad.csv", index=False)

merge_table.head()

,countyname,currstatus,FIPS
0,Alameda County,84.838517,06001
1,Amador County,89.750000,06005
2,Butte County,84.470930,06007
3,Calaveras County,91.747619,06009
4,Colusa County,89.169565,06011


In [101]:
import plotly.plotly as py
import plotly.figure_factory as ff

colorscale = ["BF0009","C32200","C75000","CB8100","CFB300","C0D400","92D800","63DC00","31E000","00E501"]    
    
endpts = list(np.linspace(merge_table["currstatus"].min(), merge_table["currstatus"].max(), len(colorscale) - 1))
fips = merge_table["FIPS"]
values = merge_table["currstatus"]

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['California'], binning_endpoints=endpts, colorscale=colorscale,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, show_state_data=False, show_hover=True,  
    centroid_marker={'opacity': 0}, asp=2.9, title='High School Graduation Rate 2016', legend_title='Graduation Rate')

py.iplot(fig, filename='ca_grad_rate')